# 1. Test Random Environment with OpenAI Gym

In [ ]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [ ]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state
    

In [644]:
__credits__ = ["Desmond N.A. Hammond"]

from os import path
from typing import Optional

import numpy as np

import gym
from gym import spaces
from gym.envs.classic_control import utils
from gym.error import DependencyNotInstalled
from numba import jit

DEFAULT_X = np.pi
DEFAULT_Y = 1.0


class PMSMEnv(gym.Env):

    metadata = {
        "render_modes": ["human", "rgb_array"],
        "render_fps": 30,
    }

    def __init__(self, render_mode: Optional[str] = None, parameters: Optional[dict] = None):

        if parameters is None:
            self.Rs = np.array(0.5) 
            self.Ld = np.array(3.5e-3)
            self.Lq = np.array(5e-3)
            self.p = np.array(3.)
            self.psi_f = np.array(0.33)
            self.Bm = np.array(0.0028)
            self.J = np.array(0.004)
            self.max_speed = np.array(500.)
            self.max_torque = np.array(10.)
            self.max_voltage = np.array(800.)
            self.max_current = np.array(500.)
            self.dt = np.array(0.001)
            self.simTime = np.array(1.)
        else:
            self.Rs = np.array(parameters.get("Rs")) if "Rs" in parameters else np.array(0.5) 
            self.Ld = np.array(parameters.get("Ld")) if "Ld" in parameters else np.array(3.5e-3)
            self.Lq = np.array(parameters.get("Lq")) if "Lq" in parameters else np.array(5e-3)
            self.p = np.array(parameters.get("p")) if "p" in parameters else np.array(3.)
            self.psi_f = np.array(parameters.get("psi_f")) if "psi_f" in parameters else np.array(0.33)
            self.Bm = np.array(parameters.get("Bm")) if "Bm" in parameters else np.array(0.0028)
            self.J = np.array(parameters.get("J")) if "J" in parameters else np.array(0.004)
            self.max_speed = np.array(parameters.get("max_speed")) if "max_speed" in parameters else np.array(500.)
            self.max_torque = np.array(parameters.get("max_torque")) if "max_torque" in parameters else np.array(10.)
            self.max_voltage = np.array(parameters.get("max_voltage")) if "max_voltage" in parameters else np.array(800.)
            self.max_current = np.array(parameters.get("max_current")) if "max_current" in parameters else np.array(500.)  
            self.dt = np.array(parameters.get("sample_time")) if "sample_time" in parameters else np.array(0.001)
            self.simTime = np.array(parameters.get("simulation_time")) if "simulation_time" in parameters else np.array(1.) 
            
        self.render_mode = render_mode

        min_action = np.array([-self.max_voltage, -self.max_voltage], dtype=np.float32)
        max_action = np.array([self.max_voltage, self.max_voltage], dtype=np.float32)

        min_observation = np.array([-self.max_current, -self.max_current, -self.max_speed], dtype=np.float32)
        max_observation = np.array([self.max_current, self.max_current, self.max_speed], dtype=np.float32)

        self.action_space = spaces.Box(low=min_action, high=max_action, shape=(2,), dtype=np.float32)
        self.observation_space = spaces.Box(low=min_observation, high=max_observation, shape=(3,), dtype=np.float32)

        # State variables
        self.state = None
        self.reset()
        
    
    def rungekutta4_step(self, diff_eqn, x0, dt):
        k1 = diff_eqn(x0)
        k2 = diff_eqn(x0 + k1 * dt/2.)
        k3 = diff_eqn(x0 + k2 * dt/2.)
        k4 = diff_eqn(x0 + k3 * dt)
        return x0 + (dt/6.) * (k1 + 2*k2 + 2*k3 + k4)
        
    def step(self, u, id_ref, omega_ref, T_load):
        
        self.T_load = np.clip(T_load, -self.max_torque, self.max_torque)
        self.vd, self.vq = np.clip(u, -self.max_voltage, self.max_voltage).tolist()
        self.x_prev = np.array([self.id,self.iq,self.omega])
        
        # Compute new states
        self.id, self.iq, self.omega = self.rungekutta4_step(diff_eqn=self.Xdot, x0=self.x_prev, dt=self.dt).tolist()
        self.x_current = np.array([self.id,self.iq,self.omega])
        self.id_dot, self.iq_dot, self.omega_dot = self.Xdot(self.x_current).tolist()
        self.lambda_d = self.Ld*self.id + self.psi_f
        self.lambda_q = self.Lq*self.iq
        self.Te = (3/2)*(self.p)*(self.lambda_d*self.iq - self.lambda_q*self.id)
        
        if self.render_mode == "human":
            self.render()
        
        # reduce time by one stimestep
        self.simTime -= self.dt

        # calculate error
        self.id_error = self.id - np.array(id_ref)
        self.omega_error = self.omega - np.array(omega_ref)

        # calculate error
        reward = -np.abs( self.id_error) -np.abs(self.omega_error)

        # check if episode is done
        done = self.simTime<=0

        # set placeholder for info
        info = {}

        return self.get_observations, reward, done, info

    
    def Xdot(self, xdot):
        id_, iq_, omega_ = xdot.tolist()

        x_dot = np.array([self.vd/self.Ld - (self.Rs*id_)/self.Ld + (self.Lq*iq_*omega_*self.p)/self.Ld,
                         self.vq/self.Lq - (self.Rs*iq_)/self.Lq - (omega_*self.p*self.psi_f)/self.Lq - (self.Ld*id_*omega_*self.p)/self.Lq,
                         -(self.T_load + self.Bm*omega_ - (3*self.p*(iq_*(self.psi_f + self.Ld*id_) - self.Lq*id_*iq_))/2)/self.J])
            
        id_dot, iq_dot, omega_dot = x_dot.tolist()
        
        return np.array([id_dot, iq_dot, omega_dot])


    def reset(self, *, seed: Optional[int] = None, options: Optional[dict] = None):
        super().reset(seed=seed)
        
        # reset state variables
        if options is None:
            self.id_dot = np.array([0.])
            self.iq_dot = np.array([0.])
            self.omega_dot = np.array([0.])            
            self.id = np.array([0.])
            self.iq = np.array([0.])
            self.omega = np.array([0.])
        else:
            self.id = options.get("id_init") if "id_init" in options else np.array([0.])
            self.iq = options.get("iq_init") if "iq_init" in options else np.array([0.])
            self.omega = options.get("omega_init") if "omega_init" in options else np.array([0.])
            self.id_dot = np.array([0.])
            self.iq_dot = np.array([0.])
            self.omega_dot = np.array([0.])

        if self.render_mode == "human":
            self.render()
        return self.get_observations(), {}

    def get_observations(self):
        # sensor measurement noise can be implemented here
         observations = np.array([self.id, self.iq, self.omega], dtype=np.float32)
         return observations
   
    def render(self):
        # Implement visualization
        pass
        
    def close(self):
        pass

In [645]:
env = PMSMEnv(
    parameters = {            
                    "Rs" : 0.5,
                    "Ld" : 3.5e-3,
                    "Lq" : 5e-3,
                    "p" : 3.,
                    "psi_f" : 0.33,
                    "Bm" : 0.0028,
                    "J" : 0.004,
                    "max_speed" : 500.,
                    "max_torque" : 10.,
                    "max_voltage" : 800.,
                    "max_current" : 500.,
                    "dt" : 0.001,
                    "simTime" : 1.
                }
)

In [646]:
env.observation_space.sample()

array([ 83.52703, 186.6295 , 237.83838], dtype=float32)

In [647]:
env.action_space.sample()

array([ 514.06647, -620.6124 ], dtype=float32)

In [648]:
# kp_i = 100;
# ki_i = 4000;

# fsat_d = 1e-3;
# fsat_q = 1e-3;


# kp_s = 1;
# ki_s = 3;

# %observer design
# Q = [1 0 0 0;
#      0 1 0 0;
#      0 0 100 0;
#      0 0 0 100;];

# R = diag([1 1]); %[1 0;0 1;];

# C = [1 0 0 0;
#     0 1 0 0;];

    
# P_init = 1e-3*eye(4)

In [649]:
episodes = 10
for episode in range(1, episodes+1):
    initial_conditions = {  "id_init": np.array([0.]),
                            "iq_init": np.array([3.]),                            
                            "omega_init": np.array([0.])
                         }
    state = env.reset(options=initial_conditions)
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        id_ref = 0
        omega_ref = 2
        T_load = 0

        n_state, reward, done, info = env.step(action, id_ref, omega_ref, T_load)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:[-407464.39942882]
Episode:2 Score:[-50.64487472]
Episode:3 Score:[-105.85865422]
Episode:4 Score:[-59.75702142]
Episode:5 Score:[-170.04962043]
Episode:6 Score:[-9.19056428]
Episode:7 Score:[-75.95068399]
Episode:8 Score:[-141.38930657]
Episode:9 Score:[-210.51024847]
Episode:10 Score:[-121.18931116]


# 2. Create a Deep Learning Model with Keras

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
states = env.observation_space.shape
actions = env.action_space.n

In [ ]:
actions

In [ ]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
del model 

In [ ]:
model = build_model(states, actions)

In [ ]:
model.summary()

# 3. Build Agent with Keras-RL

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

In [ ]:

# state matrix
a = g/(lp*(4.0/3 - mp/(mp+mk)))
A = np.array([[0, 1, 0, 0],
              [0, 0, a, 0],
              [0, 0, 0, 1],
              [0, 0, a, 0]])

# input matrix
b = -1/(lp*(4.0/3 - mp/(mp+mk)))
B = np.array([[0], [1/mt], [0], [b]])

In [ ]:

R = np.eye(2, dtype=int)          # choose R (weight for input)
Q = 5*np.eye(4, dtype=int)        # choose Q (weight for state)

# get riccati solver
from scipy import linalg

# solve ricatti equation
P = linalg.solve_continuous_are(A, B, Q, R)

# calculate optimal controller gain
K = np.dot(np.linalg.inv(R),
           np.dot(B.T, P))

In [ ]:
def apply_state_controller(K, x):
    # feedback controller
    u = -np.dot(K, x)   # u = -Kx
    if u > 0:
        return 1, u     # if force_dem > 0 -> move cart right
    else:
        return 0, u     # if force_dem <= 0 -> move cart left

In [ ]:

# get environment
env = gym.make('CartPole-v0')
env.env.seed(1)     # seed for reproducibility
obs = env.reset()

for i in range(1000):
    env.render()
    
    # get force direction (action) and force value (force)
    action, force = apply_state_controller(K, obs)
    
    # absolute value, since 'action' determines the sign, F_min = -10N, F_max = 10N
    abs_force = abs(float(np.clip(force, -10, 10)))
    
    # change magnitute of the applied force in CartPole
    env.env.force_mag = abs_force

    # apply action
    obs, reward, done, _ = env.step(action)
    if done:
        print(f'Terminated after {i+1} iterations.')
        break

env.close()